In [1]:
import os
USE_HPC=False
if USE_HPC:
    mycache_dir="/data/cmpe249-fa23/Huggingfacecache"
    os.environ['TRANSFORMERS_CACHE'] = mycache_dir
    os.environ['HF_HOME'] = mycache_dir
    os.environ['HF_DATASETS_CACHE'] = mycache_dir
    os.environ['http_proxy'] = "http://172.16.1.2:3128"
    os.environ['HTTP_PROXY'] = "http://172.16.1.2:3128"
    os.environ['https_proxy'] = "https://172.16.1.2:3128"
    os.environ['HTTPS_PROXY'] = "https://172.16.1.2:3128"
    trainoutput="/data/cmpe249-fa23/trainoutput/huggingface"
    taskname="Ms_Marcoasksciencemodeling"
else:
    trainoutput="./output"
    taskname="Ms_Marcoasksciencemodeling"

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
from datasets import load_dataset

# Load the train split and limit to the first 5000 examples
gastro = load_dataset("gayanin/pubmed-gastro-maskfilling", split="train[:5000]")

# Display a sample
print(gastro)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


masked_pubmed_highlights_dataset.csv:   0%|          | 0.00/2.50M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11772 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'highlights', 'masked_highlights'],
    num_rows: 5000
})


In [4]:
gastro = gastro.train_test_split(test_size=0.2)

In [5]:
gastro["train"][0]

{'id': 20395926,
 'highlights': 'Histopathological alterations in post-infectious irritable bowel syndrome in Bangladeshi population.',
 'masked_highlights': 'Histopathological <mask> in post-infectious irritable bowel syndrome in <mask> population.'}

In [6]:
#extract the text subfield from its nested structure with the flatten method:
gastro = gastro.flatten()
gastro

DatasetDict({
    train: Dataset({
        features: ['id', 'highlights', 'masked_highlights'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['id', 'highlights', 'masked_highlights'],
        num_rows: 1000
    })
})

In [8]:
!pip install transformer

ERROR: Could not find a version that satisfies the requirement transformer (from versions: none)
ERROR: No matching distribution found for transformer


In [9]:
from transformers import AutoTokenizer
modelname="distilroberta-base"
if USE_HPC:
    localpath=os.path.join(mycache_dir, modelname)
    tokenizer = AutoTokenizer.from_pretrained(localpath)
else:
    tokenizer = AutoTokenizer.from_pretrained(modelname)#, cache_dir=mycache_dir)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [10]:
tokenizer(gastro["train"][0]['masked_highlights'])

{'input_ids': [0, 38657, 43671, 9779, 50264, 11, 618, 12, 37597, 6514, 26570, 868, 29928, 14115, 11, 50264, 1956, 4, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [11]:
examples=gastro["train"]

In [12]:
len(examples)

4000

In [13]:
listexamples = [" ".join(x) for x in examples["highlights"]]

In [14]:
token_train=tokenizer(listexamples)

In [15]:
token_train=tokenizer(listexamples, padding='max_length', truncation=True)

In [16]:
class TokenizerWrapper:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def tokenize_function(self, examples):
        return self.tokenizer(
            [" ".join(x) for x in examples["masked_highlights"]],
            padding="max_length",
            truncation=True,
        )

In [17]:
tokenizer_wrapper = TokenizerWrapper(tokenizer)

In [18]:
tokenized_dataset = gastro.map(tokenizer_wrapper.tokenize_function, batched=True, num_proc=3, remove_columns=gastro["train"].column_names)

Map (num_proc=3):   0%|          | 0/4000 [00:00<?, ? examples/s]

Map (num_proc=3):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [19]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [20]:
def group_texts(examples):
    block_size = 128
    # Concatenate all texts.
    #print(examples.keys())
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    #print('total_length:', total_length)
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    return result

In [21]:
lm_dataset = tokenized_dataset.map(group_texts, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/4000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [22]:
lm_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 16000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 4000
    })
})

In [23]:
from transformers import DataCollatorForLanguageModeling

#tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [24]:
from transformers import AutoModelForMaskedLM
if USE_HPC:
    localpath=os.path.join(mycache_dir, modelname) #modelname="distilroberta-base"
    model = AutoModelForMaskedLM.from_pretrained(localpath)
else:
    model = AutoModelForMaskedLM.from_pretrained(modelname)#"distilroberta-base")

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
text = "The cat sat on the <mask> and looked out the window."

In [26]:
inputs = tokenizer(text, return_tensors="pt")

In [27]:
import torch
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_index

tensor([6])

In [28]:
logits = model(**inputs).logits

In [29]:
logits

tensor([[[35.4072, -3.5215, 18.2982,  ...,  3.3059,  6.2721, 13.8052],
         [ 7.3808, -3.3088, 15.1967,  ..., -0.0806,  3.0439,  5.4410],
         [-5.9852, -6.0205,  3.5735,  ..., -6.3578, -3.7101, -4.5748],
         ...,
         [-6.4342, -5.2334,  4.6863,  ..., -2.8971, -1.4126, -2.2540],
         [18.5802, -4.8933, 19.0049,  ...,  0.4924,  3.0811,  5.3561],
         [12.4169, -5.2762, 28.0920,  ..., -1.4719, -2.6244,  6.8120]]],
       grad_fn=<ViewBackward0>)

In [30]:
logits.shape

torch.Size([1, 14, 50265])

In [31]:
mask_token_logits = logits[0, mask_token_index, :]
mask_token_logits

tensor([[-5.7454, -5.0918,  2.5561,  ..., -5.9833, -5.2245, -1.6839]],
       grad_fn=<IndexBackward0>)

In [32]:
mask_token_logits.shape

torch.Size([1, 50265])

In [33]:
top_3_tokens = torch.topk(mask_token_logits, 3, dim=1).indices[0].tolist()

In [34]:
top_3_tokens

[16433, 26711, 20609]

In [35]:
for token in top_3_tokens:
    print(text.replace(tokenizer.mask_token, tokenizer.decode([token])))

The cat sat on the  couch and looked out the window.
The cat sat on the  sofa and looked out the window.
The cat sat on the  porch and looked out the window.


In [43]:
print(type(gastro))
print(gastro['train'][:5])  # Display the first few entries

<class 'datasets.dataset_dict.DatasetDict'>
{'id': [20395926, 17662670, 20962437, 12654129, 20224145], 'highlights': ['Histopathological alterations in post-infectious irritable bowel syndrome in Bangladeshi population.', 'Accuracy of upper gastrointestinal swallow study in identifying strictures after laparoscopic gastric bypass surgery.', 'Efficacy of Rikkunshito, a traditional Japanese medicine (Kampo), in treating functional dyspepsia.', 'The effects of supraphysiological doses of corticosteroids in hypotensive liver failure.', 'Food-based strategies to modulate the composition of the intestinal microbiota and their associated health effects.'], 'masked_highlights': ['Histopathological <mask> in post-infectious irritable bowel syndrome in <mask> population.', 'Accuracy of upper gastrointestinal swallow study in <mask> <mask> after laparoscopic <mask> <mask> surgery.', '<mask> of Rikkunshito, a <mask> Japanese <mask> (Kampo), in treating functional dyspepsia.', 'The effects <mask> s

In [36]:
!pip install nltk rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=cf64a3aaa8de2c9db28dd5f9550a3f71ee04a898ef5aa13f58166f09603d7e49
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [52]:
from transformers import AutoTokenizer, RobertaForMaskedLM
import torch
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer

def evaluate_model(model, tokenizer, dataset):
    bleu_scores = []
    rouge_scores = []

    # Create a Rouge scorer
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    # Iterate through the dataset
    for i in range(len(dataset['test']["masked_highlights"])):
        masked_sentence = dataset['test']["masked_highlights"][i]
        true_sentence = dataset['test']["highlights"][i]

        # Tokenize the input
        inputs = tokenizer(masked_sentence, return_tensors="pt")
        input_ids = inputs.input_ids

        # Get model outputs
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits

        # Replace the masked tokens with predicted tokens
        mask_token_index = (input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
        predicted_tokens = torch.argmax(logits[0, mask_token_index], dim=-1)

        # Decode the predicted sentence
        predicted_sentence = masked_sentence
        for idx, token in zip(mask_token_index.tolist(), predicted_tokens):
            predicted_sentence = predicted_sentence.replace(tokenizer.mask_token, tokenizer.decode([token]), 1)

        # Compute BLEU score
        bleu_score = sentence_bleu([true_sentence.split()], predicted_sentence.split())
        bleu_scores.append(bleu_score)

        # Compute ROUGE scores
        rouge = scorer.score(true_sentence, predicted_sentence)
        rouge_scores.append(rouge)

        # Print progress every 10 sentences
        if i % 10 == 0:
            print(f"Processed {i}/{len(dataset['test']['masked_highlights'])} sentences...")

    return bleu_scores, rouge_scores

# Example usage with RobertaForMaskedLM and AutoTokenizer
# model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(modelname)
model = RobertaForMaskedLM.from_pretrained(modelname)

# Evaluate the model
bleu_scores, rouge_scores = evaluate_model(model, tokenizer, gastro)

# Compute average BLEU and ROUGE scores
avg_bleu = sum(bleu_scores) / len(bleu_scores)
avg_rouge1 = sum(r["rouge1"].fmeasure for r in rouge_scores) / len(rouge_scores)
avg_rouge2 = sum(r["rouge2"].fmeasure for r in rouge_scores) / len(rouge_scores)
avg_rougeL = sum(r["rougeL"].fmeasure for r in rouge_scores) / len(rouge_scores)

print(f"Average BLEU Score: {avg_bleu}")
print(f"Average ROUGE-1 F1: {avg_rouge1}")
print(f"Average ROUGE-2 F1: {avg_rouge2}")
print(f"Average ROUGE-L F1: {avg_rougeL}")

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Processed 0/1000 sentences...
Processed 10/1000 sentences...


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Processed 20/1000 sentences...
Processed 30/1000 sentences...
Processed 40/1000 sentences...
Processed 50/1000 sentences...


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Processed 60/1000 sentences...
Processed 70/1000 sentences...
Processed 80/1000 sentences...


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Processed 90/1000 sentences...
Processed 100/1000 sentences...
Processed 110/1000 sentences...
Processed 120/1000 sentences...
Processed 130/1000 sentences...
Processed 140/1000 sentences...
Processed 150/1000 sentences...
Processed 160/1000 sentences...
Processed 170/1000 sentences...
Processed 180/1000 sentences...
Processed 190/1000 sentences...
Processed 200/1000 sentences...
Processed 210/1000 sentences...
Processed 220/1000 sentences...
Processed 230/1000 sentences...
Processed 240/1000 sentences...
Processed 250/1000 sentences...
Processed 260/1000 sentences...
Processed 270/1000 sentences...
Processed 280/1000 sentences...
Processed 290/1000 sentences...
Processed 300/1000 sentences...
Processed 310/1000 sentences...
Processed 320/1000 sentences...
Processed 330/1000 sentences...
Processed 340/1000 sentences...
Processed 350/1000 sentences...
Processed 360/1000 sentences...
Processed 370/1000 sentences...
Processed 380/1000 sentences...
Processed 390/1000 sentences...
Processed

In [54]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
import math

def compute_perplexity(model, tokenizer, text):
    """
    Compute perplexity for a text using a masked language model.
    """
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt")
    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask

    # Mask a random token (excluding special tokens)
    mask_token_index = torch.where(input_ids[0] == tokenizer.mask_token_id)[0]
    if len(mask_token_index) == 0:
        raise ValueError("Text does not contain a mask token. Ensure you include '<mask>' in the input.")

    # Get model outputs
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    # Calculate probabilities for the masked token
    log_probs = 0
    for idx in mask_token_index:
        token_logits = logits[0, idx]
        target_token_id = input_ids[0, idx]
        token_prob = torch.softmax(token_logits, dim=0)[target_token_id]
        log_probs += torch.log(token_prob)

    # Average the log probabilities
    avg_log_prob = log_probs / len(mask_token_index)

    # Compute perplexity
    perplexity = torch.exp(-avg_log_prob)
    return perplexity.item()

# Example usage
# model_name = "distilroberta-base"
# tokenizer = AutoTokenizer.from_pretrained(modelname)
# model = AutoModelForMaskedLM.from_pretrained(modelname)

# Example text
text = "The quick brown fox <mask> over the lazy dog."
perplexity = compute_perplexity(model, tokenizer, text)
print(f"Perplexity: {perplexity}")

Perplexity: 765994.4375


In [46]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=os.path.join(trainoutput, modelname, taskname), #"./output/my_awesome_eli5_mlm_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=10,
    weight_decay=0.01,
    push_to_hub=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [47]:
from huggingface_hub import notebook_login
notebook_login()

In [48]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

In [49]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.927000,0.824044
2,0.767700,0.669845
3,0.690200,0.583996
4,0.602100,0.564821
5,0.511200,0.504574
6,0.529800,0.469129
7,0.511100,0.450599
8,0.481400,0.434290
9,0.472500,0.413990
10,0.423800,0.421397


TrainOutput(global_step=20000, training_loss=0.6255485397338867, metrics={'train_runtime': 7032.1354, 'train_samples_per_second': 22.753, 'train_steps_per_second': 2.844, 'total_flos': 1.053061853184e+16, 'train_loss': 0.6255485397338867, 'epoch': 10.0})

In [50]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 1.51


In [57]:
import pickle

# Save the model
with open("trained_model.pkl", "wb") as f:
    pickle.dump(model, f)

# # Load the model
# with open("trained_model.pkl", "rb") as f:
#     loaded_model = pickle.load(f)

In [56]:
# Save the model using Hugging Face API
output_dir = os.path.join(trainoutput, modelname, taskname)
trainer.save_model("/content/saved_model")  # Saves the model, tokenizer, and training args

# Verify the save
print("Model saved")

No files have been modified since last commit. Skipping to prevent empty commit.


Model saved


In [58]:
with open('trained_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

In [59]:
loaded_model

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm)